In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from statsmodels.tsa.api import Holt, SimpleExpSmoothing

In [2]:
sys.path.insert(1, '../')
from helpers.decision_maker import DecisionMaker
from seeds import known_seeds
from evaluation import get_actual_demand
from evaluation_output import evaluate
from utils import save_solution
from utils import load_problem_data

In [3]:
def get_my_solution(d, p, seed):
    _, datacenters, servers, selling_prices = load_problem_data(path='../data/')
    
    decision_maker = DecisionMaker(datacenters,servers,selling_prices, d, seed, p)

    return decision_maker.solve()

In [4]:
def get_all_solutions(p, seed):


    demand = pd.read_csv('../data/demand.csv')
    
    # SET THE RANDOM SEED
    np.random.seed(seed)

    # GET THE DEMAND
    actual_demand = get_actual_demand(demand)

    # CALL YOUR APPROACH HERE
    solution = get_my_solution(actual_demand, p, seed)

    # SAVE YOUR SOLUTION
    save_solution(solution, f'../output/{seed}.json')

In [ ]:
ps = [i/100 for i in range(15, 31, 1)]
seeds = [2311, 3329, 
                4201, 
                8761,  
                2663, 
                4507, 
                6247, 
                2281, 
                4363, 
                5693]



all_best_params = {}

for seed in seeds:
    best_params = {}

    params = []
    for i in range(len(ps)):
        
        p = random.choice(ps)
        while(p in params):
            p = random.choice(ps)
        params.append(p)
        get_all_solutions(p,seed)
        
        avg = evaluate(seed, '../')
        f = open('results.txt', 'a+')
        f.write("SEED#: " + str(seed) + " | score: " + str(avg) + " | p: " +  str(p)  + "\n")
        f.close()
        if len(best_params)<5:
            best_params[avg] = p
        else:
            worst_score = sorted(best_params.keys())[0]
            if avg > worst_score:
                del best_params[worst_score]
                best_params[avg] = p

    print("SEED#: ", str(seed))
    print(best_params)
    all_best_params[seed] = best_params

